Part 1: Code Review & Debugging
#
@app.route('/api/products', methods=['POST'])
def create_product():
    data = request.json
     
    # Create new product
    product = Product(
        name=data['name'],
        sku=data['sku'],
        price=data['price'],
        warehouse_id=data['warehouse_id']
    )
     
    db.session.add(product)
    db.session.commit()
     
    # Update inventory count
    inventory = Inventory(
        product_id=product.id,
        warehouse_id=data['warehouse_id'],
        quantity=data['initial_quantity']
    )
     
    db.session.add(inventory)
    db.session.commit()
     
    return {"message": "Product created", "product_id": product.id}
Tasks:
1. Identify Issues: List all problems you see with this code (technical and business
logic)
2. Explain Impact: For each issue, explain what could go wrong in production
3. Provide Fixes: Write the corrected version with explanations

**Solution**

Technical Issues
1. Lack of input validation

- No checks for required fields or incorrect data types.

- Price should be validated as a decimal, initial_quantity as non-negative integer.

2. SKU uniqueness not enforced

- SKUs must be unique across the platform, but there’s no query to check before inserting.

3. Multiple warehouses support ignored

- The code assumes a product belongs to a single warehouse at creation.

4. Two separate commits

- Product and Inventory entries are committed in separate transactions, which may cause partial save issues if the second commit fails.

5. No error handling

- If DB commit fails, no rollback; could leave inconsistent data.

6. No handling of optional fields

- All fields are assumed to exist in data.

Business Logic Issues

1. Products in multiple warehouses

- Should check if the product already exists for another warehouse.

2. Initial stock not mandatory

- Should allow creating a product without inventory.

3. Decimal precision for price

- Ensure correct DB column type (e.g., DECIMAL(10,2)).

Impact in Production
1. Duplicate SKUs → data integrity issues.

2. Missing validation → crashes or wrong data stored.

3. Partial commits → ghost products without inventory.

4. No error handling → poor API reliability.

In [ ]:
# Fixed Code

@app.route('/api/products', methods=['POST'])
def create_product():
    data = request.get_json()

    # Validate required fields
    required_fields = ['name', 'sku', 'price', 'warehouse_id']
    for field in required_fields:
        if field not in data:
            return {"error": f"Missing field: {field}"}, 400

    # Check SKU uniqueness
    existing_product = Product.query.filter_by(sku=data['sku']).first()
    if existing_product:
        return {"error": "SKU already exists"}, 409

    try:
        # Create product
        product = Product(
            name=data['name'],
            sku=data['sku'],
            price=float(data['price'])
        )
        db.session.add(product)
        db.session.flush()  # Get product.id before commit

        # Add inventory if provided
        initial_quantity = data.get('initial_quantity', 0)
        inventory = Inventory(
            product_id=product.id,
            warehouse_id=data['warehouse_id'],
            quantity=initial_quantity
        )
        db.session.add(inventory)

        db.session.commit()
        return {"message": "Product created", "product_id": product.id}, 201

    except Exception as e:
        db.session.rollback()
        return {"error": str(e)}, 500


**Part 2: Database Design**

Based on the requirements below, design a database schema.

Given Requirements:
- Companies can have multiple warehouses
- Products can be stored in multiple warehouses with different quantities
- Track when inventory levels change
- Suppliers provide products to companies
- Some products might be "bundles" containing other products

Tasks:
1. Design Schema: Create tables with columns, data types, and relationships
2. Identify Gaps: List questions you'd ask the product team about missing
requirements
3. Explain Decisions: Justify your design choices (indexes, constraints, etc.)

In [ ]:
-- Companies
CREATE TABLE companies (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL
);

-- Warehouses
CREATE TABLE warehouses (
    id SERIAL PRIMARY KEY,
    company_id INT NOT NULL REFERENCES companies(id),
    name VARCHAR(255) NOT NULL
);

-- Products
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    sku VARCHAR(100) UNIQUE NOT NULL,
    price DECIMAL(10,2) NOT NULL,
    product_type VARCHAR(50), -- for low-stock threshold rules
    is_bundle BOOLEAN DEFAULT FALSE
);

-- Inventory
CREATE TABLE inventory (
    product_id INT REFERENCES products(id),
    warehouse_id INT REFERENCES warehouses(id),
    quantity INT DEFAULT 0,
    PRIMARY KEY (product_id, warehouse_id)
);

-- Inventory History
CREATE TABLE inventory_history (
    id SERIAL PRIMARY KEY,
    product_id INT REFERENCES products(id),
    warehouse_id INT REFERENCES warehouses(id),
    change_amount INT NOT NULL,
    change_type VARCHAR(50), -- sale, restock, etc.
    change_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Suppliers
CREATE TABLE suppliers (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    contact_email VARCHAR(255)
);

-- Product-Supplier relationship
CREATE TABLE product_suppliers (
    product_id INT REFERENCES products(id),
    supplier_id INT REFERENCES suppliers(id),
    PRIMARY KEY (product_id, supplier_id)
);

-- Bundle items
CREATE TABLE bundle_items (
    bundle_id INT REFERENCES products(id),
    product_id INT REFERENCES products(id),
    quantity INT NOT NULL,
    PRIMARY KEY (bundle_id, product_id)
);


Gaps & Questions to Ask
- How is product type determined for thresholds?

- Do suppliers serve multiple companies or are they company-specific?

- Should we store pricing history?

- How are bundles priced — fixed or sum of items?

- Do we track units sold for sales activity?

Design Decisions
- Indexes on sku, warehouse_id for fast lookups.

- Composite keys for many-to-many tables.

- inventory_history for full audit trail.

- Separate bundle table to support kit products.

**Part 3: API Implementation**

Implement an endpoint that returns low-stock alerts for a company.

Business Rules (discovered through previous questions):

- Low stock threshold varies by product type
-  Only alert for products with recent sales activity
-  Must handle multiple warehouses per company
-  Include supplier information for reordering


Endpoint Specification:
GET /api/companies/{company_id}/alerts/low-stock





In [ ]:
Expected Response Format:
{
  "alerts": [
    {
      "product_id": 123,
      "product_name": "Widget A",
      "sku": "WID-001",
      "warehouse_id": 456,
      "warehouse_name": "Main Warehouse",
      "current_stock": 5,
      "threshold": 20,
      "days_until_stockout": 12,
      "supplier": {
        "id": 789,
        "name": "Supplier Corp",
        "contact_email": "orders@supplier.com"
      }
    }
  ],
  "total_alerts": 1
}

Tasks:
1. Write Implementation: Use any language/framework (Python/Flask,
Node.js/Express, etc.)
2. Handle Edge Cases: Consider what could go wrong
3. Explain Approach: Add comments explaining your logic

In [ ]:
# Flask Implementation


@app.route('/api/companies/<int:company_id>/alerts/low-stock', methods=['GET'])
def get_low_stock_alerts(company_id):
    try:
        # Join inventory with products, warehouses, suppliers
        query = db.session.query(
            Products.id.label('product_id'),
            Products.name.label('product_name'),
            Products.sku,
            Warehouses.id.label('warehouse_id'),
            Warehouses.name.label('warehouse_name'),
            Inventory.quantity.label('current_stock'),
            Thresholds.threshold,
            Suppliers.id.label('supplier_id'),
            Suppliers.name.label('supplier_name'),
            Suppliers.contact_email
        ).join(Inventory, Inventory.product_id == Products.id)\
         .join(Warehouses, Inventory.warehouse_id == Warehouses.id)\
         .join(ProductSuppliers, ProductSuppliers.product_id == Products.id)\
         .join(Suppliers, Suppliers.id == ProductSuppliers.supplier_id)\
         .join(Thresholds, Thresholds.product_type == Products.product_type)\
         .filter(Warehouses.company_id == company_id)\
         .filter(Inventory.quantity < Thresholds.threshold)\
         .filter(Products.id.in_(
             db.session.query(Sales.product_id)
             .filter(Sales.date >= datetime.utcnow() - timedelta(days=30))
         ))

        alerts = []
        for row in query:
            # Estimate days until stockout
            avg_daily_sales = get_avg_daily_sales(row.product_id, row.warehouse_id)
            days_until_stockout = (
                row.current_stock / avg_daily_sales if avg_daily_sales > 0 else None
            )
            alerts.append({
                "product_id": row.product_id,
                "product_name": row.product_name,
                "sku": row.sku,
                "warehouse_id": row.warehouse_id,
                "warehouse_name": row.warehouse_name,
                "current_stock": row.current_stock,
                "threshold": row.threshold,
                "days_until_stockout": days_until_stockout,
                "supplier": {
                    "id": row.supplier_id,
                    "name": row.supplier_name,
                    "contact_email": row.contact_email
                }
            })

        return {"alerts": alerts, "total_alerts": len(alerts)}

    except Exception as e:
        return {"error": str(e)}, 500


Edge Cases
- No sales in last 30 days → exclude product.

- Multiple suppliers → pick primary or return all.

- Zero sales rate → cannot estimate days until stockout.